# 1.0 Fix data

AIMS 

En este archivo se realiza la limpieza de los archivos y el ETL

In [4]:
import pandas as pd
from mypackage import dir

In [5]:
%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [6]:
modality =  modality = 'p'
project = 'australia'
data = dir.make_dir_line(modality, project) 
raw = data('raw')

In [7]:
df = pd.read_csv(raw / 'df_clientes.csv', sep = ',', decimal = '.', header = 0)
df.head()

,id_cliente,nombre,sexo,fecha_nacimiento
0,1001,Maria Casaleggio,F,1991-04-29
1,1002,Sig.ra Rosalia Serlupi,F,2015-08-10
2,1003,Claudia Toninelli-Endrizzi,F,2012-02-09
3,1004,Eugenia Palazzo,F,2008-04-04
4,1005,Sig.ra Lucrezia Pratesi,F,1958-09-22
